In [64]:
from misc_functions            import *
from db.Database import *
# import db.Area as Area
from db.AreaExit import *
import json

In [65]:
def get_exits_from_area_id(area_id):
    exits = []
    try:
        exits = AreaExit.select().where((AreaExit.area_from_id == area_id))
    except AreaExit.DoesNotExist:
        exits = []
    return exits

area_id = 2
initial_exits = get_exits_from_area_id(area_id)

In [66]:
def get_exit_json(exits):
    # we wanna make some json edges and nodes/areas for cytoscape from our los map data
    # let's start simple and look out 2x steps from teh start area (Chapel areaid 2)
    edges = []

    for exit in initial_exits:
        # we should append json of the edge id, source (area_from_id), target (area_to_id)
        # skip if the area_to_id is null
        if exit.area_to_id is not None:
            edges.append({"id": exit.id, "source": exit.area_from_id, "source_name": exit.area_from.name, "target": exit.area_to_id, "target_name": exit.area_to.name})

    return edges

In [67]:
def get_areas_json(exits):
    areas = {}
    for exit in initial_exits:
        # we should append json of the edge id, source (area_from_id), target (area_to_id)
        # skip if the area_to_id is null
        areas[exit.area_from_id] = {"id": exit.area_from_id, "name": exit.area_from.name}
        
        if exit.area_to_id is not None:
            areas[exit.area_to_id] = {"id": exit.area_to_id, "name": exit.area_to.name}

    return areas

In [68]:
elements = []
areas_mapped = [area_id]

def get_new_json(exits):
    elements = []
    edges = get_exit_json(exits)
    areas = get_areas_json(exits)

    # for cytoscape, we need to have a list of nodes and a list of edges as json elements with data attributes
    for area in areas:
        nice_name = f'{areas[area]["id"]}_{areas[area]["name"]}' 
        elements.append({"data": {"id": areas[area]["id"], "label": nice_name}})

    for edge in edges:
        nice_name = f'{edge["id"]}_{edge["source_name"]}_{edge["target_name"]}'
        elements.append({"data": {"id": nice_name, "source": edge["source"], "target": edge["target"]}})
    return elements

elements = get_new_json(initial_exits)
for exit in initial_exits:
    if exit.area_to_id is not None and exit.area_to_id not in areas_mapped:
        area_id = exit.area_to_id
        exits = get_exits_from_area_id(area_id)
        elements += get_new_json(exits)
        areas_mapped.append(area_id)

    

print(json.dumps(elements))

[{"data": {"id": 2, "label": "2_The Chapel of Healing"}}, {"data": {"id": 3, "label": "3_Holly Lane"}}, {"data": {"id": 6, "label": "6_Prayer Chamber"}}, {"data": {"id": "2_The Chapel of Healing_Holly Lane", "source": 2, "target": 3}}, {"data": {"id": "4_The Chapel of Healing_Prayer Chamber", "source": 2, "target": 6}}]
